In [ ]:
#k-NN with Tsfresh
#検知したデータを用いてk-NN(100個学習100個推定)
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from pylab import rcParams
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
import changefinder
import seaborn as sns
from tsfresh import extract_features, extract_relevant_features
from tsfresh.feature_selection.selection import select_features
from tsfresh.utilities.dataframe_functions import impute
import glob

In [ ]:
with tf.device('/device:GPU:0'): #GPU使用
    
    def make_all_ppg_array(data,original_data):#120秒分の脈波を取り出し
        time=0
        for line in original_data:

            if time < 12000 :
                data[time] = int(line.split()[3])#何列目とってくるか
                time += 1
                
        return data


    def Anomaly_Detection(ppg,answer,label,data_type):

        data =np.zeros((12000),int)#データ格納用
        det_data = np.zeros((0,det_data_len),int)#検知データ格納用
        sub_det_data = np.zeros((det_data_len),int) #切り出した検知データ格納用
        det_data_num = 0 #データ数を数えるため


        data = make_all_ppg_array(data,ppg)

        #確認のためとりあえず描画
        plt.plot(data, linestyle = "dashed" , label="acc")
        plt.xlabel('Epochs',fontsize = 18)
        plt.ylabel('Accuracy',fontsize = 18)
        plt.legend()
        plt.tick_params()
        plt.show()

        cf = changefinder.ChangeFinder(r=0.01, order=1, smooth=7)#変化点検知にChangeFinder使用

        score = np.zeros((12000),int)#異常度格納用
        score_num = 0 #score index
        for i in data:
            sub_score = cf.update(i)
            score[score_num] = sub_score
            score_num += 1

        
        det_time=0 #sub_det_data index
        wait_F = 0 #ジェスチャ終了時の異常度の山を無視するため
        wait_time = 0
        for t in range(0,len(score)):
            if score[t] > 60 and wait_time == 0 and t>2000: #20秒以降で異常度の閾値を60に
                print(f"record:{t}")
                for k in range(t-100,t+100):
                    sub_det_data[det_time] = data[k]
                    det_time += 1
                det_time = 0
                det_data = np.r_[det_data,sub_det_data.reshape(1,-1)] #det_dataに検知したデータを追加
                answer_list.append(answer)
                label_list.append(label)
                wait_time = 1
                if data_type == "train" :
                    det_data_num += 1
            elif wait_time != 0 :
                wait_time += 1
                if wait_time > 500 : #5秒間何もしない
                    wait_time = 0

        print(det_data) #確認用



        # 異常度をプロット
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.plot(score)
        ax2 = ax.twinx()
        ax2.plot(data,'r')
        plt.show()
        
        return det_data,det_data_num

    det_data_len = 200 #切り出すデータの長さ
    get_data = np.zeros((0,det_data_len),int)
    train_data_num = 0
    data_num = 0
    answer_list = [] #ジェスチャ名格納用
    label_list = []  #ジェスチャのラベル格納用
    train_label_list = []
    test_label_list = []
    file_gesture = ["gu","choki","par","one","three","four","fox","phone","good","koyubi"]
    
    for i in range(len(file_gesture)):
        for file in glob.glob(f'train_data/{i}*'):
            print(file)
            with open(file) as ppg :
                new_data , data_num = Anomaly_Detection(ppg,file_gesture[i],i,"train")
                get_data = np.r_[get_data,new_data]
                train_data_num += data_num
    
    for i in range(len(file_gesture)):
        print(file)
        for file in glob.glob(f'test_data/{i}*'):
            with open(file) as ppg :
                new_data , data_num = Anomaly_Detection(ppg,file_gesture[i],i,"test")
                get_data = np.r_[get_data,new_data]
        
    train_label_list = label_list[:train_data_num]
    test_label_list = label_list[train_data_num:]
    
    np_label_list = np.array(label_list,dtype=int)   #numpy配列に変換
    np_train_label_list = np.array(train_label_list,dtype=int)
    np_test_label_list = np.array(test_label_list,dtype=int)
    
    print(answer_list)
    print(get_data)
    print(get_data.shape)
    
    #Tsfresh用にDataFrameに変換
    get_data_df = pd.DataFrame(get_data)
    
    Ts_get_data_df = pd.DataFrame({0: get_data_df[:len(get_data)].values.flatten(),
                                   1: np.arange(len(get_data)).repeat(get_data_df.shape[1])})
    
    get_data_features_df = extract_features(Ts_get_data_df, column_id=1)   #特徴量抽出
    
    get_data_features_df = impute(get_data_features_df)    #Nanなどを補間
    
    get_data_features_df=get_data_features_df.drop(columns=get_data_features_df.select_dtypes(include='object').columns)     #数値以外の物を除去
    
    print(get_data_features_df)
    print(np_label_list.shape)
    selected_get_data_fea = select_features(get_data_features_df,np_label_list)
    
    get_data_features = selected_get_data_fea.values
    
    train_data_features = get_data_features[:train_data_num,:]
    test_data_features = get_data_features[train_data_num:,:]
        
    print(train_data_features.shape)
    print(test_data_features.shape)

    
    #k分割交差検証
    kf = StratifiedKFold(n_splits=5, shuffle=True,random_state=0)

    for train_index, eval_index in kf.split(train_data_features,np_train_label_list):
        X_train, X_test = train_data_features[train_index], train_data_features[eval_index]
        Y_train, Y_test = np_train_label_list[train_index], np_train_label_list[eval_index]
        
        accuracy_list = []
        sns.set()
        k_range = range(1, 9)
        for k in k_range: #kの値を変えながらk-NNの精度検証
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, Y_train)
            Y_pred = knn.predict(X_test)
            test_result = knn.predict(test_data_features)
            accuracy_list.append(metrics.accuracy_score(Y_test, Y_pred))
            print(f"k={k}:validation")
            print(confusion_matrix(Y_test,Y_pred)) #混同行列を作成
            print(f"k={k}:test")
            print(confusion_matrix(test_label_list,test_result))
            print(metrics.accuracy_score(Y_test,Y_pred))
            print(metrics.accuracy_score(np_test_label_list,test_result))
        
        #精度の変化を描画
        figure = plt.figure()
        ax = figure.add_subplot(111)
        ax.plot(k_range, accuracy_list)
        ax.set_xlabel('k-nn')
        ax.set_ylabel('accuracy')
        plt.show()